<a href="https://colab.research.google.com/github/pollinations/hive/blob/main/interesting_notebooks/FiLM%20-%20Frame%20Interpolation%20for%20Large%20Motion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### FiLM: Frame Interpolation for Large Motion

This notebook is a convenience wrapper around a pre-trained model released by ML researchers at Google. Learn more about their work here: 

* https://github.com/google-research/frame-interpolation

---

Notebook authored by David Marx ([@DigThatData](https://twitter.com/DigThatData)), released under the MIT license.

If you experience any issue with or have suggestions regarding this notebook, please report them here:
* https://github.com/pytti-tools/pytti-notebook/issues/new 


In [11]:

# Upload a zip file containing images or a video file
image_file = ''  #@param {type: "string"}

# If you start with $n$ frames in `frames_dir` and set `recursive_interpolation_passes` to $k$, your total number of frames after interpolation will be: 2^k (n-1) -1
recursive_interpolation_passes = 1 #@param {type: "number"}

# Frame rate of output video
output_video_fps = 30 #@param {type: "number"}

#@markdown ---

output_path = "/content"

In [12]:
!mkdir -p /content/frames
if image_file.endswith('.zip'):
  !unzip -o "{image_file}" -d /content/frames
else:
  !ffmpeg -i "{image_file}" /content/frames/frame%5d.jpg

Archive:  /content/Archive 3.zip
  inflating: /content/frames/progress_00185.jpeg  
  inflating: /content/frames/__MACOSX/._progress_00185.jpeg  
  inflating: /content/frames/progress_00030.jpeg  
  inflating: /content/frames/__MACOSX/._progress_00030.jpeg  
  inflating: /content/frames/progress_000001.jpeg  
  inflating: /content/frames/__MACOSX/._progress_000001.jpeg  


In [13]:
# This top bit only neneds to be run once
# ...may as well set up the notebook to 
# assume the user is only going to run all
# of setup once.
from pathlib import Path
import os

drive_mounted = False
gdrive_fpath = '.'
local_path = '/content/'

##############################

#@markdown # 1. Setup Workspace

#@markdown Run this cell to perform setup.

#@markdown Mounting your google drive is optional.
#@markdown If you mount your drive, code and models will be downloaded to it.
#@markdown This should reduce setup time after your first run.

###################

# Optionally Mount GDrive

mount_gdrive = False # @param{type:"boolean"}
if mount_gdrive and not drive_mounted:
    from google.colab import drive

    gdrive_mountpoint = '/content/drive/' #@param{type:"string"}
    gdrive_subdirectory = 'MyDrive/interpolation' #@param{type:"string"}
    gdrive_fpath = str(Path(gdrive_mountpoint) / gdrive_subdirectory)
    try:
        drive.mount(gdrive_mountpoint, force_remount = True)
        !mkdir -p {gdrive_fpath}
        %cd {gdrive_fpath}
        local_path = gdrive_fpath
        drive_mounted = True
    except OSError:
        print(
            "\n\n-----[PYTTI-TOOLS]-------\n\n"
            "If you received a scary OSError and your drive"
            " was already mounted, ignore it."
            "\n\n-----[PYTTI-TOOLS]-------\n\n"
            )
        raise

#####################

# Perform rest of setup

if not Path('./frame-interpolation').exists():
    !git clone https://github.com/pytti-tools/frame-interpolation

try:
    import frame_interpolation
except ModuleNotFoundError:
    %pip install -r ./frame-interpolation/requirements_colab.txt
    %pip install ./frame-interpolation

#url = "https://drive.google.com/drive/folders/1GhVNBPq20X7eaMsesydQ774CgGcDGkc6?usp=sharing"
share_id = "1GhVNBPq20X7eaMsesydQ774CgGcDGkc6"

if not (Path(local_path) / 'saved_model').exists():
    !pip install --upgrade gdown
    !gdown --folder {share_id}

# create default frame
!mkdir -p frames

In [14]:
#@markdown # 2. Setup Interpolation

#@markdown Specify the directory containing your video frames with the `frames_dir` parameter.

frames_dir = "/content/frames" #@param{'type':'string'}

#@markdown A single pass of the interpolation procedure adds a frame between each contiguous pair of frames in `frames_dir`.


#@markdown Check this box to generate a video output. If no output video will be generated, the FPS option can be ignored.
output_video = True #@param{'type':'boolean'}


In [15]:
#@markdown # 3. Interpolate!
#@markdown Results will be saved in a subdirectory of `frames_dir` named "interpolated_frames"

from loguru import logger

logger.info("Beginning interpolation...")

if output_video:
  !python -m frame_interpolation.eval.interpolator_cli \
      --model_path ./saved_model \
      --pattern {frames_dir} \
      --fps {output_video_fps} \
      --output_video
      

else:
    !python -m frame_interpolation.eval.interpolator_cli \
      --model_path ./saved_model \
      --pattern {frames_dir} \

logger.info("Interpolation comlpete.")

2022-03-09 13:57:37.350 | INFO     | __main__:<module>:6 - Beginning interpolation...


2022-03-09 13:57:40.686 | DEBUG    | __main__:_run_pipeline:171 - ['/content/frames']
W0309 13:57:40.761598 140220736698240 environments.py:374] Make sure that locally built Python SDK docker image has Python 3.7 interpreter.
I0309 13:57:40.761775 140220736698240 environments.py:380] Default Python SDK image for environment is apache/beam_python3.7_sdk:2.34.0
I0309 13:57:40.788255 140220736698240 translations.py:678] ==================== <function annotate_downstream_side_inputs at 0x7f871076c8c0> ====================
I0309 13:57:40.788503 140220736698240 translations.py:678] ==================== <function fix_side_input_pcoll_coders at 0x7f871076c9e0> ====================
I0309 13:57:40.788650 140220736698240 translations.py:678] ==================== <function pack_combiners at 0x7f871076cef0> ====================
I0309 13:57:40.788948 140220736698240 translations.py:678] ==================== <function lift_combiners at 0x7f871076cf80> ====================
I0309 13:57:40.789077 140220

2022-03-09 13:58:38.541 | INFO     | __main__:<module>:14 - Interpolation comlpete.


In [16]:
!cp -v /content/frames/interpolated.mp4 {output_path}

'/content/frames/interpolated.mp4' -> '{output_path}'


```
MIT License

Copyright (c) 2022 David Marx

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```